# Simple Example of Palars, Dask, DuckDB 

## Setup

Install the dependencies if you haven't already.

In [1]:
# !pip install -r requirements.txt

In [2]:
from datetime import date

import polars as pl
import dask.dataframe as dd
import duckdb

## Example 1: Read And Query Parquet files

References:

- Polars: https://pola-rs.github.io/polars/user-guide/io/parquet/
- Dask: https://docs.dask.org/en/latest/dataframe-parquet.html
- DuckDB: https://duckdb.org/docs/data/parquet/overview.html

### Polars

Read and query Parquet files

In [3]:
pl_df = pl.read_parquet("../data/2023/parquet/*.parquet")
pl_df.filter((pl.col("date") == pl.lit(date(2023, 12, 22))) & (pl.col("close") >= 1.5))

date,ticker,close
date,str,f64
2023-12-22,"""intu""",1.595353
2023-12-22,"""phm""",2.2171


Lazy evaluation using Polars

In [4]:
pl_lazy_df = pl.scan_parquet("../data/2023/parquet/*.parquet")
pl_lazy_df.filter((pl.col("date") == pl.lit(date(2023, 12, 22))) & (pl.col("close") >= 1.5)).collect()

date,ticker,close
date,str,f64
2023-12-22,"""intu""",1.595353
2023-12-22,"""phm""",2.2171


### Dask

Read and query Parquet files

In [5]:
dd_df = dd.read_parquet("../data/2023/parquet/*.parquet")
dd_df[(dd_df["date"] == date(2023, 12, 22)) & (dd_df["close"] >= 1.5)].compute()

,date,ticker,close
245,2023-12-22,intu,1.595353
245,2023-12-22,phm,2.217100


### DuckDB

Read and query Parquet files

In [6]:
TABLE = "../data/2023/parquet/*.parquet"
duckdb.query(f"""SELECT * FROM '{TABLE}' WHERE date = '2023-12-22' AND close >= 1.5""")

┌────────────┬─────────┬────────────────────┐
│    date    │ ticker  │       close        │
│    date    │ varchar │       double       │
├────────────┼─────────┼────────────────────┤
│ 2023-12-22 │ phm     │  2.217099567099567 │
│ 2023-12-22 │ intu    │ 1.5953525231351298 │
└────────────┴─────────┴────────────────────┘

## Example 2: Pivot and Aggregate

### Polars

Pivot and aggregate

In [7]:
pl_df = pl.read_parquet("../data/2023/parquet/*.parquet")
pl_final_df = pl_df.pivot(index="ticker", columns="date", values="close").with_columns(
    pl.concat_list(pl.all().exclude("ticker")).alias("closes")
)
pl_final_df.write_parquet("./tmp/polars.parquet")

### Dask

Pivot and aggregate

In [8]:
dd_df = dd.read_parquet("../data/2023/parquet/*.parquet")

# Make date column to categorical to use pivot method
dd_df.date = dd_df.date.dt.strftime("%Y-%m-%d").astype("category").cat.as_known()

dd_pivot_df = dd_df.pivot_table(index='ticker', columns='date', values='close').compute()

# Aggregate the date columns
dd_agg_df = dd_df.compute().groupby('ticker').agg({"close": list})

# Merge the two dataframes
dd_final_df = dd_pivot_df.merge(dd_agg_df, on="ticker")

dd_final_df.rename(columns={"close": "closes"}, inplace=True)

dd_final_df.to_parquet("./tmp/dask.parquet")

**NOTE: Object data type conversion issue**

Above example works fine, "closes" column data type is list of float64

In [9]:
type(dd_final_df.closes.iloc[0])

list

In [10]:
type(dd_final_df.closes.iloc[0][0])

float

However, if you move lazy evaluation to the end, the "closes" column data type is converted to string

In [11]:
dd_dtissue_df = dd.read_parquet("../data/2023/parquet/*.parquet")

# Make date column to categorical to use pivot method
dd_dtissue_df.date = dd_dtissue_df.date.dt.strftime("%Y-%m-%d").astype("category").cat.as_known()

dd_dtissue_pivot_df = dd_dtissue_df.pivot_table(index='ticker', columns='date', values='close')

# Aggregate the date columns
dd_dtissue_agg_df = dd_dtissue_df.groupby('ticker').agg({"close": list})

# Merge the two dataframes
dd_dtissue_final_df = dd_dtissue_pivot_df.merge(dd_dtissue_agg_df, on="ticker").compute()

dd_dtissue_final_df.rename(columns={"close": "closes"}, inplace=True)

dd_dtissue_final_df.to_parquet("./tmp/dask.parquet")

In [12]:
dd_dtissue_final_df.closes.iloc[0][:120]

'[1.0, 1.0108637696614236, 1.0137963209810719, 0.9842042122100773, 0.98287123433751, 1.0345907757931219, 1.05418555051986'

Dask community is aware of this issue, see below links for more details.

- [[FEEDBACK] User experience with arrow strings](https://github.com/dask/dask/issues/10139)
- [Issues with `convert-string` label](https://github.com/dask/dask/issues?q=label%3Aconvert-string+)

### DuckDB

Pivot and aggregate

In [13]:
TABLE = "../data/2023/parquet/*.parquet"

# Directly join the results of the two queries using subqueries
duckdb.sql(f"""
CREATE OR REPLACE TEMPORARY TABLE final_t AS (
    SELECT pivot_t.*, agg_t.closes
    FROM (
        PIVOT '{TABLE}' ON date USING first(close) GROUP BY ticker
    ) AS pivot_t
    INNER JOIN (
        SELECT ticker, list(close ORDER BY date ASC) AS closes 
        FROM '{TABLE}' 
        GROUP BY ticker
    ) AS agg_t ON pivot_t.ticker = agg_t.ticker
)
""")

In [14]:
duckdb.sql("SELECT * FROM final_t").to_df().head()

,ticker,2023-01-03,2023-01-04,2023-01-05,2023-01-06,2023-01-09,2023-01-10,2023-01-11,2023-01-12,2023-01-13,...,2023-12-12,2023-12-13,2023-12-14,2023-12-15,2023-12-18,2023-12-19,2023-12-20,2023-12-21,2023-12-22,closes
0,cnc,1.0,0.996489,0.968899,0.986456,0.973037,0.954602,0.983697,0.980436,0.966140,...,0.957361,0.961500,0.946953,0.930775,0.937923,0.934663,0.905568,0.920617,0.919739,"[1.0, 0.9964885879107098, 0.9688989214948585, ..."
1,duk,1.0,1.010704,0.995757,1.014272,1.014368,1.016683,1.018901,1.013308,1.012343,...,0.919286,0.945805,0.948023,0.931823,0.937030,0.933558,0.922662,0.929219,0.934041,"[1.0, 1.0107039537126326, 0.9957569913211186, ..."
2,cah,1.0,1.003519,0.998566,1.024892,1.010947,1.018246,0.997263,0.997263,1.001825,...,1.406621,1.408185,1.338329,1.309657,1.317868,1.318259,1.291802,1.319432,1.320605,"[1.0, 1.0035188322689952, 0.9985664016681871, ..."
3,trmb,1.0,1.003953,0.990514,1.017787,1.032411,1.037549,1.052569,1.069960,1.077273,...,0.950395,0.972727,1.023320,1.023123,1.020751,1.036364,1.016206,1.034783,1.036364,"[1.0, 1.0039525691699605, 0.9905138339920948, ..."
4,pru,1.0,1.017089,0.999598,1.015380,0.987736,0.998693,1.005227,1.001206,1.015279,...,1.027141,1.046844,1.057600,1.044029,1.037093,1.048854,1.030458,1.037696,1.040812,"[1.0, 1.0170888620828307, 0.9995979091274627, ..."


Export to Parquet

In [15]:
OUTPUT = "./tmp/duckdb.parquet"

duckdb.sql(f"""COPY final_t TO '{OUTPUT}' (FORMAT PARQUET)""")